In [1]:
import numpy as np
import torch
from torch import nn
import torchvision
from torchvision.transforms import v2
from tqdm import tqdm


In [2]:
transform = v2.Compose([v2.ToTensor(),
     v2.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


/home/vadimpp/.local/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [3]:
class Cifar_model(nn.Module):
    def __init__(self,in_channels,batch_size=32,output_size=10):
        super(Cifar_model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=5)
        self.b_norm1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        self.l_relu = nn.LeakyReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.b_norm2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.fc1 = nn.Linear(in_features=5*5*64,out_features=256)
        self.fc2 = nn.Linear(in_features=256,out_features=64)
        self.fc3 = nn.Linear(in_features=64, out_features=output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self,x):

        x = self.conv1(x)
        x = self.b_norm1(x)
        x = self.l_relu(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.b_norm2(x)
        x = self.l_relu(x)
        x = self.pool2(x)
    
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)

        return x     

In [4]:
model = Cifar_model(in_channels=3)

In [5]:
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(params=model.parameters(), lr=3e-4)

In [6]:
epochs = 5
model.train()
for ep in range(epochs):
    lm_count = 0
    loss_mean = 0.0 
    tqdm_loader = tqdm(trainloader, leave=True)
    for (x,y) in tqdm_loader:
        pred = model(x)
        loss = criterion(pred,y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        lm_count += 1
        loss_mean = 1/lm_count * loss.item() + (1 - 1/lm_count) * loss_mean
        tqdm_loader.set_description(f"Epoch [{ep+1}/{epochs}], loss_mean={loss_mean:.3f}")

Epoch [5/5], loss_mean=0.689: 100%|█████████| 1563/1563 [00:30<00:00, 51.31it/s]


In [11]:
Q = 0

model.eval()

for x_test, y_test in testloader:
    with torch.no_grad():
        p = model(x_test)
        p = torch.argmax(p, dim=1)
        #print(p[0],y_test[0])
        Q += torch.sum(p == y_test).item()
 
Q = Q/len(testset)
print(Q)

0.7261
